An arbitrary two dimensinoal non-linear problem (taken from http://stackoverflow.com/questions/33135238/how-to-solve-this-system-of-two-equations-in-sympy)

In [ ]:
%matplotlib inline
import sympy as sp
sp.init_printing()
import numpy as np
import matplotlib.pyplot as plt
from pyneqsys.symbolic import SymbolicSys
def f(y, params):
    eq = lambda x, fx: 1/(x+y[0])**params[0] + y[1] - fx
    return [eq(0, 1), eq(1, 0)]
neqsys = SymbolicSys.from_callback(f, 2, 1)
neqsys.names = 'a b'.split()
neqsys.exprs

In [ ]:
params = 1
ab, sol = neqsys.solve('scipy', [0.5, -0.5], params)
ab, sol.success

In [ ]:
varied_data = np.linspace(.5, 3)
parameter_id = 0
xres, sols = neqsys.solve_series('scipy', [.5, -.5], params, varied_data, parameter_id)

In [ ]:
neqsys.plot_series(xres, varied_data)
plt.legend()
fig = plt.gcf()

If we want the plot to be interactive we may use bokeh:

In [ ]:
try:
    from bokeh.plotting import figure, output_notebook, show
    from bokeh.mpl import to_bokeh
    output_notebook()
    show(to_bokeh(fig))
except ImportError:
    print('bokeh not installed')

That was easy, let's use this system of equations to explore different algortihms.

# Custom solvers
We can also use a custom solver, there are examples of simple (not production quality) solvers in ``pyneqsys.solvers``

In [ ]:
def solve_custom(solver, plot_attr=None):
    ncols = 3 if plot_attr is None else 4
    ab, sol = neqsys.solve(solver, [0.5, -0.5], [1], tol=1e-5)
    x_history = np.array(sol['solver_cb'].history_x)
    plt.figure(figsize=(15, 3))
    plt.subplot(1, ncols, 1)
    plt.plot(x_history[:, 0], x_history[:, 1]); plt.xlabel('x0'), plt.ylabel('x1')
    plt.subplot(1, ncols, 2)
    plt.plot(neqsys.rms(x_history, [1])); plt.xlabel('iteration'), plt.ylabel('RMS(residuals)')
    plt.subplot(1, ncols, 3)
    plt.semilogy(range(15, len(x_history)), neqsys.rms(x_history[15:], [1])); plt.xlabel('iteration'), plt.ylabel('RMS(residuals)')
    if plot_attr is not None:
        plt.subplot(1, ncols, 4)
        plt.plot(getattr(solver, plot_attr))
        plt.ylabel(plot_attr)
        plt.xlabel('iteration')
    plt.tight_layout()

Let's start with Gradient descent

In [ ]:
from pyneqsys.solvers import DampedGradientDescentSolver
solve_custom(DampedGradientDescentSolver(1, 0))  # Undamped

That didn't go too well, the name of the class may give a hint of a possible solution. Let's damp the steps:

In [ ]:
solve_custom(DampedGradientDescentSolver(.5, .5))  # (under-)damped

In [ ]:
solve_custom(DampedGradientDescentSolver(.05, .05))  # over-damped

In [ ]:
solve_custom(DampedGradientDescentSolver(.3, .1))   # (close to) optimally  damped

So we should strive to limit the osciallatory behaviour, let's see if we can achieve that algorithmically:

In [ ]:
from pyneqsys.solvers import AutoDampedGradientDescentSolver
solve_custom(AutoDampedGradientDescentSolver(.1, .1, 8, .3), 'history_damping')

In [ ]:
solve_custom(AutoDampedGradientDescentSolver(.1, .1, 6, .3), 'history_damping')

In [ ]:
solve_custom(AutoDampedGradientDescentSolver(.1, .1, 4, .3), 'history_damping')

In [ ]:
solve_custom(AutoDampedGradientDescentSolver(.1, .1, 2, .3), 'history_damping')